In [1]:
#Esto me sirve para el correcto funcionamiento de las funciones importadas en este notebook
%load_ext autoreload
%autoreload 2

# Preparando los datos

Cargamos las librerías que se usarán en esta sección:

In [2]:
from utils.workit import crear_df_jugador
from utils.paths import crear_funcion_directorio
from utils.workit import eliminar_columnas_comunes
from utils.workit import generar_df_mvp
import functools
import pandas as pd

## Leemos los archivos

Iniciamos seleccionando cuatro conjuntos de datos obtenidos [**NBA Stats (1947-present)**](https://www.kaggle.com/datasets/sumitrodatta/nba-aba-baa-stats) que contienen información sobre jugadores a lo largo de todas las temporadas de la NBA. A continuación las razones de su elección:

*   **df_premios_jugadores**: Este conjunto de datos presenta información sobre todos los premios otorgados a lo largo de la historia de la NBA, más precisamente del premio que nos importa, el premio **MVP de la temporada regular**.

*   **estadisticas_avanzadas**: Contiene una amplia variedad de estadísticas **avanzadas** que sirven para evaluar el rendimiento de los jugadores en una temporada. Este conjunto de datos incluye entre sus métricas más interesantes el **PER** (Player Efficiency Rating) y el impacto del jugador en la cancha.

*   **estadisticas_tiro**: Proporciona diversas estadísticas relacionada con el tiro. Entre las más importantes se encuentran mediciones sobre los tiros de 3 y de media distancia.

*   **estadisticas_por_juego**: Proporciona promedios por partido de diversas estadísticas para un jugador. Estos datos permiten tener una aproximación bastante buena sobre lo que fue la contribución del jugador en cada partido.

In [3]:
#Creo un acceso directo a la carpeta de data
data_dir = crear_funcion_directorio("data")

#Cargo los conjuntos de datos
df_premios_jugadores = pd.read_csv(data_dir("raw", "Player Award Shares.csv"))
estadisticas_avanzadas = pd.read_csv(data_dir("raw", "Advanced.csv"))
estadisticas_tiro = pd.read_csv(data_dir("raw", "Player Shooting.csv"))
estadisticas_por_juego = pd.read_csv(data_dir("raw", "Player Per Game.csv"))

## Construyendo el conjunto de datos

Primero damos un vistazo a las columnas de los conjuntos de daots para analizar su estructura general:

In [4]:
print("Columnas de df_premios_jugadores:\n", df_premios_jugadores.columns.tolist(), "\n")
print("Columnas de estadisticas_avanzadas:\n", estadisticas_avanzadas.columns.tolist(), "\n")
print("Columnas de estadisticas_tiro:\n", estadisticas_tiro.columns.tolist(), "\n")
print("Columnas de estadisticas_por_juego:\n", estadisticas_por_juego.columns.tolist(), "\n")

Columnas de df_premios_jugadores:
 ['season', 'award', 'player', 'age', 'tm', 'first', 'pts_won', 'pts_max', 'share', 'winner', 'seas_id', 'player_id'] 

Columnas de estadisticas_avanzadas:
 ['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age', 'experience', 'lg', 'tm', 'g', 'mp', 'per', 'ts_percent', 'x3p_ar', 'f_tr', 'orb_percent', 'drb_percent', 'trb_percent', 'ast_percent', 'stl_percent', 'blk_percent', 'tov_percent', 'usg_percent', 'ows', 'dws', 'ws', 'ws_48', 'obpm', 'dbpm', 'bpm', 'vorp'] 

Columnas de estadisticas_tiro:
 ['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age', 'experience', 'lg', 'tm', 'g', 'mp', 'fg_percent', 'avg_dist_fga', 'percent_fga_from_x2p_range', 'percent_fga_from_x0_3_range', 'percent_fga_from_x3_10_range', 'percent_fga_from_x10_16_range', 'percent_fga_from_x16_3p_range', 'percent_fga_from_x3p_range', 'fg_percent_from_x2p_range', 'fg_percent_from_x0_3_range', 'fg_percent_from_x3_10_range', 'fg_percent_from_x10_16_ran

Podemos observar que las columnas **'seas_id', 'season', 'player_id' y 'player'** funcionan como **identificadores** en nuestros conjuntos de datos. Por otro lado, hay algunas **columnas en común** más allá de los identificadores, por lo tanto se procede a eliminarlas para evitar redundancia:

In [5]:
#Guardo en una variable las columnas que son identificadores
identificadores = ['seas_id', 'season', 'player_id','player']

#Tomo a 'estadisticas_tiro' y a 'estadisticas_por_juego' como los conjuntos de datos que voy a modificar
estadisticas_tiro = eliminar_columnas_comunes(df_a_modificar = estadisticas_tiro, df_sin_modificar = estadisticas_avanzadas, columnas_ignorar= identificadores)
estadisticas_por_juego = eliminar_columnas_comunes(df_a_modificar= estadisticas_por_juego, df_sin_modificar= estadisticas_avanzadas, columnas_ignorar= identificadores)
estadisticas_por_juego = eliminar_columnas_comunes(df_a_modificar= estadisticas_por_juego, df_sin_modificar= estadisticas_tiro, columnas_ignorar= identificadores)

En el caso de **df_premios_jugadores**, como se dijo anteriormente, este proyecto tratará sobre el **premio MVP** y de este conjunto de datos la variable que nos va a servir para nuestro objetivo será la columna **'mvp_share'**, que trae información sobre la proporción de votos que recibió un jugador para el premio. Por lo tanto se procede a tomar solo dicha columna:

In [6]:
#Me quedo con las instancias que traen información solo del premio al MVP
df_premios_jugadores = df_premios_jugadores[df_premios_jugadores["award"]=="nba mvp"]

#Eliminamos las columnas que no usaremos
df_premios_jugadores = df_premios_jugadores.drop(['award','age','tm','first','pts_won','pts_max','winner'], axis = 1)
df_premios_jugadores.rename(columns={'share': 'mvp_share'}, inplace=True)

Ahora que realizamos los ajustes necesarios lo ideal sería tener **toda esta información en un único conjunto de datos**, mismo que se llamará **'df_data_jugadores'**. En el mismo además se usará **información desde el año 2000 hasta el 2023** para acotar el análisis a la NBA actual y evitar el ruido en los datos.

In [7]:
#Guardo los 4 conjuntos de datos en una lista
lista_conjuntos_datos = [estadisticas_avanzadas, estadisticas_tiro, df_premios_jugadores, estadisticas_por_juego]

#Aplico el filtro
for i, df in enumerate(lista_conjuntos_datos):
    lista_conjuntos_datos[i] = df.loc[(df['season'] >= 2000)]

#Usamos functools.reduce para aplicar la función merge() de manera acumulativa a la lista de DataFrames
df_data_jugadores = functools.reduce(lambda left, right: pd.merge(left, right, on=identificadores, how='inner' if left.empty else 'left'), lista_conjuntos_datos)

#Finalmente miramos el dataset resultante
df_data_jugadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14422 entries, 0 to 14421
Data columns (total 78 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   seas_id                        14422 non-null  int64  
 1   season                         14422 non-null  int64  
 2   player_id                      14422 non-null  int64  
 3   player                         14422 non-null  object 
 4   birth_year                     153 non-null    float64
 5   pos                            14422 non-null  object 
 6   age                            14422 non-null  float64
 7   experience                     14422 non-null  int64  
 8   lg                             14422 non-null  object 
 9   tm                             14422 non-null  object 
 10  g                              14422 non-null  int64  
 11  mp                             14422 non-null  float64
 12  per                            14417 non-null 

A partir de este output hay algunas filas y columnas que procedemos a eliminar:

* **'birth_year'** dado que tiene muchos valores faltantes y no resulta útil para el análisis
* **'experience'** ya que es una columna que se deduce a partir de 'season'
*  los equipos en la columna **'tm'** de nombre **"TOT"** que representan jugadores que fueron cambiados de equipo 
   a media temporada, esta información más allá de ser relevante genera duplicados en los datos que son díficiles de detectar

In [8]:
df_data_jugadores.drop(['birth_year'], axis = 1, inplace= True)
df_data_jugadores.drop(['experience'], axis = 1, inplace= True)
df_data_jugadores.drop(df_data_jugadores[df_data_jugadores['tm'] == 'TOT'].index, inplace=True)

Ahora que tenemos un conjunto de datos con la información que necesitamos, el siguiente paso es **seleccionar a los jugadores que servirán para el análisis** y que información queremos de ellos. Los criterios de selección de la información son los siguientes: 

* El jugador debió **recibir votos** por el premio alguna vez 
* Se va a tomar en cuenta una temporada en la que recibió votos **junto con las adyacentes a la misma**, es decir,
  **una temporada antes y una despúes de este suceso**, así también se permite analizar temporadas que fueron buenas probablemente
  pero que simplemente los jugadores no recibieron votos
* A las temporadas adyacentes que no tengan votos se les imputará **el valor 0** en la columna **'mvp_share'**, lo cual es igual a decir que no recibieron votos, todo esto 
  para evitar problemas con valores faltantes

In [9]:
#Filtro el conjunto de datos
df_data_jugadores = generar_df_mvp(df = df_data_jugadores, columna_nombre= 'player', columna_votos= 'mvp_share', columna_temporadas= 'season')

#Imputo el valor 0
df_data_jugadores['mvp_share'].fillna(0, inplace=True)

#Ahora imprimo en consola
df_data_jugadores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 585 entries, 1520 to 13218
Data columns (total 76 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   seas_id                        585 non-null    int64  
 1   season                         585 non-null    int64  
 2   player_id                      585 non-null    int64  
 3   player                         585 non-null    object 
 4   pos                            585 non-null    object 
 5   age                            585 non-null    float64
 6   lg                             585 non-null    object 
 7   tm                             585 non-null    object 
 8   g                              585 non-null    int64  
 9   mp                             585 non-null    float64
 10  per                            585 non-null    float64
 11  ts_percent                     585 non-null    float64
 12  x3p_ar                         585 non-null  

## Separamos los datos

Ya tenemos el conjunto de datos ideal, el siguiente paso es hacer la separación en entrenamiento y prueba para empezar con el análisis exploratorio. Los criterios 
que usaremos para separar los datos son los siguientes:

* Tomaremos como datos de entrenamiento las temporadas **entre 2000 y 2021**
* Tomaremos como datos de prueba las temporadas **2022 y 2023**
* Decidimos sacar del conjunto de datos de prueba a **5 jugadores que fueron ganadores del premio en temporadas recientes**, esto
  con motivo de **eliminar el sesgo** de un futuro modelo respecto a las predicciones, ya que si ve que en el conjunto de entrenamiento
  un jugador viene ganando votos por el premio, lo más probable es que en los datos de prueba también le asigne una proporción alta de votos

In [10]:
#Guardo en una lista a los jugadores que sacaré 
jugadores_seleccionados = ['Giannis Antetokounmpo', 'Joel Embiid','Nikola Jokić','LeBron James','James Harden']

#Genero sus conjuntos de datos individuales
df_giannis = crear_df_jugador('Giannis Antetokounmpo', columna_nombre= 'player', df = df_data_jugadores)
df_embiid = crear_df_jugador('Joel Embiid', columna_nombre= 'player', df = df_data_jugadores)
df_jokic = crear_df_jugador('Nikola Jokić', columna_nombre= 'player', df = df_data_jugadores)
df_harden = crear_df_jugador('James Harden', columna_nombre= 'player', df = df_data_jugadores)
df_lebron = crear_df_jugador('LeBron James', columna_nombre= 'player', df = df_data_jugadores)

#Los elimino del dataset original para ahora convertirlo en un train
df_data_jugadores_train  = df_data_jugadores[~df_data_jugadores['player'].isin(jugadores_seleccionados)]
df_data_jugadores_train  = df_data_jugadores_train[df_data_jugadores_train['season'] < 2022]

#Junto todos esos dataframes en el test que evaluaremos al final
df_data_jugadores_test  = df_data_jugadores[df_data_jugadores['season'] >= 2022]

Finalmente, guardo los conjuntos de datos **en la carpeta data**:

In [11]:
df_giannis.to_csv(data_dir("processed", "df_giannis.csv"), index=False)
df_embiid.to_csv(data_dir("processed", "df_embiid.csv"), index=False)
df_jokic.to_csv(data_dir("processed", "df_jokic.csv"), index=False)
df_harden.to_csv(data_dir("processed", "df_harden.csv"), index=False)
df_lebron.to_csv(data_dir("processed", "df_lebron.csv"), index=False)
df_data_jugadores_train.to_csv(data_dir("processed", "df_data_jugadores_train.csv"), index=False)
df_data_jugadores_test.to_csv(data_dir("processed", "df_data_jugadores_test.csv"), index=False)
df_data_jugadores.to_csv(data_dir("processed", "df_data_jugadores.csv"), index=False)